In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import streamlit as st
from sklearn.neighbors import NearestNeighbors

In [2]:
data=pd.read_csv(r'C:\Users\Gaming PC\Desktop\bepec\churn/bankProduct.csv')

In [5]:
data.head()

,user_Id,email,product,rating
0,15889,cust15889@gmail.com,Medium-term deposits,3.5
1,15890,cust15890@gmail.com,Securities,3.0
2,15891,cust15891@gmail.com,Direct Debit,4.0
3,15892,cust15892@gmail.com,Payroll Account,2.0
4,15893,cust15893@gmail.com,Pensions,1.0


In [3]:
df = data.pivot_table(index="product", columns="email", values="rating").fillna(0)

In [6]:
df

email,cust15889@gmail.com,cust15890@gmail.com,cust15891@gmail.com,cust15892@gmail.com,cust15893@gmail.com,cust15894@gmail.com,cust15895@gmail.com,cust15896@gmail.com,cust15897@gmail.com,cust15898@gmail.com,...,cust43116@gmail.com,cust43118@gmail.com,cust43123@gmail.com,cust43124@gmail.com,cust43125@gmail.com,cust43131@gmail.com,cust43132@gmail.com,cust43133@gmail.com,cust43134@gmail.com,cust43136@gmail.com
product,,,,,,,,,,,,,,,,,,,,,
Credit Card,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Current Accounts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Derivada Account,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Direct Debit,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Funds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
Guarantees,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
Home Account,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Junior Account,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
Loans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_matrix = csr_matrix(df.values)
df_matrix

<23x13091 sparse matrix of type '<class 'numpy.float64'>'
	with 13091 stored elements in Compressed Sparse Row format>

In [8]:
model_knn = NearestNeighbors(metric="cosine", algorithm="brute")
model_knn.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [9]:
def recom_system(distances, indices, index):
    pro = None
    rec = []
    for i in range(0, len(distances.flatten())):
        if i == 0:
            pro = df.index[index].values[0]
        else:
            rec.append(df.index[indices.flatten()[i]])
    return pro, rec


In [13]:
def callback():
    st.session_state.button_clicked = True

In [14]:
def home():
    st.session_state.button_clicked = False

In [15]:
def main():
    st.title("Product Recommendation System")
    st.write("Banking Products")
    if st.checkbox("Registered Users", on_change=home):
        st.write(data[["email","product","rating"]].head())
    user = st.text_input("Enter your mail")
    if st.button("Enter"):
        if user in list(data["email"].unique()):
            st.write(f"Welcome back! {user[:9].upper()}")
            index = data[data["email"]==user].index.values
            distances, indices = model_knn.kneighbors(df.iloc[index,:].values.reshape(1,-1), n_neighbors=6)
            product, rec = recom_system(distances, indices, index)
            st.write(f"your current product : ' __{product}__ '")
            st.write("#### Products you may like ")
            col1, col2, col3, col4, col5  = st.columns(5)
            with col1:
                st.write(f"__{rec[0]}__")
                st.image("https://i.gifer.com/JVX7.gif")
            with col2:
                st.write(f"__{rec[1]}__")
                st.image("https://i.gifer.com/JVX7.gif")
            with col3:
                st.write(f"__{rec[2]}__")
                st.image("https://i.gifer.com/JVX7.gif")
            with col4:
                st.write(f"__{rec[3]}__")
                st.image("https://i.gifer.com/JVX7.gif")
            with col5:
                st.write(f"__{rec[4]}__")
                st.image("https://i.gifer.com/JVX7.gif")
            st.button("home")
        else:
            st.error("User not Registered")
            st.write("create a new account or use registered email")
    else:
        if "button_clicked" not in st.session_state:
            st.session_state.button_clicked = False
        if (st.button("Create Account") or st.session_state.button_clicked):

            user_mail = st.text_input("Your Email")
            # st.session_state
            if (st.button("Sign up!", on_click=callback) or st.session_state.button_clicked):
                st.success("Successful")
                st.subheader("Welcome new user!")
                st.write("Welcome to our Bank")
                select = st.multiselect("Products Available", list(data["product"].unique()))
                # st.write("your selected product: ",select)
                st.write("#### Some popular products ")
                rec2 = list(data.loc[data["rating"]>4.9, "product"].unique())[:6]
                print(rec2)
                col1, col2, col3, col4, col5  = st.columns(5)
                with col1:
                    st.write(f"__{rec2[0]}__")
                    st.image("https://i.gifer.com/9uff.gif")
                with col2:
                    st.write(f"__{rec2[1]}__")
                    st.image("https://i.gifer.com/9uff.gif")
                with col3:
                    st.write(f"__{rec2[2]}__")
                    st.image("https://i.gifer.com/9uff.gif")
                with col4:
                    st.write(f"__{rec2[3]}__")
                    st.image("https://i.gifer.com/9uff.gif")
                with col5:
                    st.write(f"__{rec2[4]}__")
                    st.image("https://i.gifer.com/9uff.gif")
                st.balloons()
                st.button("home", on_click=home)


    st.write("Thank you for using our bank")
    st.markdown("<h1 style='text-align: right; color: #d7e3fc; font-size: small;'><a href='https://github.com/Nikhil-Jagtap619/product_recommendation'>Looking for Source Code?</a></h1>", unsafe_allow_html=True)


In [16]:
if __name__ == "__main__":
    main()

2022-07-20 16:12:58.867 
  command:

    streamlit run C:\Users\Gaming PC\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2022-07-20 16:12:58.869 Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "button_clicked". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization